# Project 2: Exploring the GitHub Dataset with Colaboratory
---


In this project, you will explore one of BigQuery's public datasets on GitHub and learn to make visualizations in order to answer your questions. This project is due on **Monday, November 1st, 11:59PM**. It is worth 50 points, for 15% of your overall grade. After completing this project, make sure to follow the submission instructions in the handout to submit on Gradescope.


### Notes (read carefully!):

* Be sure you read the instructions on each cell and understand what it is doing before running it.
* Don't forget that if you can always re-download the starter notebook from the course website if you need to.
* You may create new cells to use for testing, debugging, exploring, etc., and this is in fact encouraged! Just make sure that the final answer for each question is **in its own cell** and **clearly indicated**.
* Colab will not warn you about how many bytes your SQL query will consume.  **Be sure to check on the BigQuery UI first before running queries here!**
* See the assignment handout for submission instructions.
* Have fun!

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* *Name, SUNet ID*

# Overview

BigQuery has a massive dataset of GitHub files and statistics, including information about repositories, commits, and file contents.  In this project, we will be working with this dataset. Don't worry if you are not too familiar with Git and GitHub -- we will explain everything you need to know to complete this part of the assignment.

---
## Notes

**The GitHub dataset available on BigQuery is actually quite massive. *A single query on the "contents" table alone (it is 2.16TB!) can eat up your 1TB allowance for the month AND cut into about 10% of your GCloud credit for the class.***

To make this part of the project more manageable, we have subset the original data.  We have preserved almost all information in the original tables, but we kept only the information on the top 500,000 most "watched" GitHub repos between January 2016 and October 2018.

You can see the tables we will be working with [here](https://console.cloud.google.com/bigquery?project=cs145-fa21-326819&authuser=1&p=cs145-fa21-326819&d=project2&page=dataset). **Read through the schemas to get familiar with the data**.  Note that some of the tables are still quite large (the contents table is about 500GB), so you should exercise the usual caution when working with them.  Before running queries on this notebook, it's good practice to first set up query limits on your BigQuery account or see how many bytes will be billed on the web UI.

---

## *Make sure to use our subsetted dataset, not the original BigQuery dataset!*





## A Super Quick Primer on Git

If you are not very familiar with Git and GitHub, here are some high-level explanations that will give you enough context to get you through this part of the problem:

*  *GitHub*: GitHub is a source-control service provider.  GitHub allows you to collaborate on and keep track of source code in a fairly efficient way.

*  *commit*: A commit can be thought of as a change that is applied to some set of files. i.e., if some set of files is in some state A, you can make changes to A and *commit* your changes to the set of files so that it is now in state B.  A commit is identified by a *hash* of the information in your change (the author of the commit, who actually committed [i.e. applied] the changes to the set of files, the changes themselves, etc.)

* *parent commit*: The commit that came before your current commit.

*   *repo*:  A repo (short for repository) is GitHub's abstraction for a collection of files along with a history of commits on those files. If you have GitHub username "foo" and you make a repository called "data-rocks", your repo name will be "foo/data-rocks".  You can think of a repo's history in terms of its commits.  E.g., "foo/data-rocks" can have the set of "states" A->B->C->D, where each state change (A->B, B->C, C->D) was due to a commit.

*   *branch*:  To keep track of different commit histories, GitHub repos can have branches.  The 'main' branch (i.e. commit history) of the repo is called the 'master' branch.  Say on "foo/data-rocks" we have the commit history A->B->C->D on the master branch.  If someone else comes along and decides to add a cool new feature to "foo/data-rocks", they can create a branch called "cool-new-feature" that branches away from the master branch. All the code from the main branch will be there, but new code added to "cool-new-feature" will not be on the main branch.

*   *ref*: For the purpose of this assignment, you can think of the 'ref' field on the "files" table as referring to the branch in which a file lives in a repository at some point in time.



For the purposes of this question, you don't need to know about the following things in detail:


*   Commit trees
*   The encoding attribute on the commits table

If you want more clarifications about Git and GitHub in order to answer this question, be sure to post on Ed or come to Office Hours.  In many cases, a quick web search will also help answer your question.

# Section 1 | Understanding the Dataset (4 points)

---

Things to note:
* A file ID changes based on a file's contents; it is not assigned at a file's creation.
* Different repos can have files with the same paths.
* It is possible to have separate files with identical contents.
* A repo may have one file across multiple branches.



## Question 1: Schema Comprehension (4 points)

Each of the following parts is worth 1 point.




### a)  What is the primary key of `github_repo_files`? (1 point)

---
lrepo_name

---

### b) What is the primary key in `github_repo_licenses`? What is the foreign key? (1 point)

---
FORIEGN KEY: lrepo_name,license

PRIMARY KEY: lrepo_name	

---


### c) If we were given authors and we wanted to know what programming languages they used in the repos that they contributed to, which tables should we use? (1 point)

---

github_repo_languages and github_repo_commits

---


### d) If we wanted to know whether using different licenses had an effect on a repo's watch count, which tables would we use? (1 point)

---

github_repos

github_repo_licenses

---

# Section 2 | Query Performance (8 points)
---


In this section, we'll look at some inefficient queries and think about how we can make them more efficient. For this section, we'll consider efficiency in terms of how many bytes are processed.

## Question 2: Optimizing Queries (8 points)


For the next three subquestions, consider the following query:

```
SELECT DISTINCT author.name
FROM `cs145-fa21-326819.project2.github_repo_commits` commits_1
WHERE (SELECT COUNT(*) 
       FROM `cs145-fa21-326819.project2.github_repo_commits` commits_2
       WHERE commits_1.author.name = commits_2.author.name) > 20

```

**NOTE:** We do **NOT** recommend running this unoptimized query in BigQuery, as it will run for a very long time (over 15 minutes if not longer). However, feel free to run an optimized version of this query after finishing part (c), which takes about 5 seconds to run.

### a) In one to two sentences, explain what this query does. (1 point)

---

In this question, it will show those who have committed more than 20 times

---

### b) Briefly explain why this query is inefficient (in terms of bytes that need to be processed) and how it can be improved to be more efficient. (1 point)

---
There was no need to use subquery for this since it can be done without it

LIMIT Should be used to sample query results since the dataset used is very large. 

---

### c) Following from part (b), write a more efficient version of the query. (2 points)

---
```
SELECT DISTINCT author.name 
FROM `cs145-fa21-326819.project2.github_repo_commits`  
GROUP BY author.name
HAVING count(author.name) > 20 
```
---

For the next three subquestions, consider the following query:

```
SELECT id
FROM (
  SELECT files.id, files.mode, contents.size
  FROM
    `cs145-fa21-326819.project2.github_repo_files` files,
    `cs145-fa21-326819.project2.github_repo_readme_contents` contents
  WHERE files.id = contents.id
  )
WHERE mode = 33188 AND size > 1000  
LIMIT 10
```


###d) Briefly explain why this query is inefficient (in terms of bytes that need to be processed) without the query optimization and how it can be improved to be more efficient. (1 point)

---
This query can be optimized by simply adding the where condition in the inner query instead of two. 

---

###e) Following from part (d), write a more efficient version of the query. (2 points)

Hint: Think about the number of bytes processed by the unoptimized query. Can any operator be moved around to reduce this number?

---
```
    SELECT files.id
    FROM
    `cs145-fa20-291100.project2.github_repo_files` files,
    `cs145-fa20-291100.project2.github_repo_readme_contents` contents
    WHERE files.id = contents.id and mode = 33188 AND size > 1000
```
---

###f) Run both the original query and your optimized query on BigQuery and pay attention to the number of bytes processed. How do they compare, and is it what you expect? Explain why this is happening in a few sentences. (1 point)

Hint: Look at the query plan under "Execution details" in the bottom panel of BigQuery. It may be especially helpful to look at stage "S00: Input".

---

When you run a duplicate query, BigQuery attempts to reuse cached results. To retrieve data from the cache, the duplicate query text must be the same as the original query.

---

To learn more about writing efficient SQL queries and how BigQuery optimizes queries, check out [Optimizing query computation](https://cloud.google.com/bigquery/docs/best-practices-performance-compute) and [Query plan and timeline](https://cloud.google.com/bigquery/query-plan-explanation).